# Read image stack

In [ ]:
import imageio
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

## Settings

In [ ]:
dpi_x = 600
dpi_y = 300
dz    = 27e-4 # 27 micron
# directory = '/Users/birnstiel/Desktop/slices_pluto_color/'
directory = 'slices_fit3/'

## Read Images

Get number of files and file dimensions

In [ ]:
files = sorted(list(Path(directory).glob('*.png')))
im = imageio.imread(files[0]).transpose(1, 0, 2)
_x = np.arange(im.shape[0])
_y = np.arange(im.shape[1])

dx = 2.54 / dpi_x
dy = 2.54 / dpi_y
dpi_z = 2.54 / dz

Sum up all filled pixels: this counts all pixels, that are not `[255, 255, 255]`, so other than white.

In [ ]:
summed_image = np.zeros(im.shape[:2])

for file in tqdm(files):
    summed_image += np.array(imageio.imread(file).sum(-1) < 3 * 255, dtype=int).T

## Print out statistics

In [ ]:
print(f'{len(files)} files')
print(f'dimension = {im.shape[0] * dx:.2f} x {im.shape[1] * dy:.2f} x {len(files) * dz:.2f} cm')
print(f'nr of empty pixels: {(summed_image==0).sum() / np.prod(summed_image.shape):.2%}')
print(f'most opaque pixel has {summed_image.max():n} filled pixels (={summed_image.max() / len(files):.2%} of all layers are filled)')

## Plots

### Statistics plots
Show fully transparent pixels and the density distribution.

In [ ]:
f, axs = plt.subplots(1, 3, figsize=(15, 4))

ax = axs[0]
cc=ax.imshow(summed_image>0, cmap='gray_r')
ax.set_aspect(dpi_y / dpi_x)
plt.colorbar(cc, ax=ax)
ax.set_title('white = all voxels transparent');

ax = axs[1]
ax.set_aspect(dpi_x / dpi_y)
i = ax.pcolormesh(_x, _y, summed_image.T, vmin=0, vmax=summed_image.max(), cmap='gray_r')
cb = plt.colorbar(i, ax=ax)
cb.set_label('# of opaque pixes along LOS')

ax = axs[-1]
counts, bins, patches = ax.hist(summed_image.ravel(), bins=np.arange(summed_image.max()))
ax.set_xlabel('number of filled voxels in column')
ax.set_xlabel('count');

### Compare data to 3d-printed result

In [ ]:
slice_width = .1 # in cm
n_slice = int(slice_width / dz)

In [ ]:
img = np.zeros(im.shape[:2])

for i, file in enumerate(tqdm(files[:n_slice])):
    img += np.array(imageio.imread(file).sum(-1) < 3 * 255, dtype=int).T

In [ ]:
f, axs = plt.subplots(1, 3, dpi=200, figsize=(15,5))
axs[0].imshow(img.T / img.max(), cmap='gray_r')
axs[0].set_aspect(dpi_x / dpi_y)
axs[0].set_title(f'sum over first {slice_width:.2f} cm')

axs[1].imshow(imageio.imread('results/img1.jpg'))
axs[1].set_title('45º front illuminated')
axs[2].axis('off')

axs[2].imshow(imageio.imread('results/img2.jpg'))
axs[2].set_title('side illuminated')
axs[2].axis('off');

In [ ]:
from IPython.display import Video
Video('results/turbulent_box.mp4', width=500, html_attributes='autoplay loop')